In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
nba = pd.read_csv('https://raw.githubusercontent.com/jkropko/contrans/main/examples/ASA%20All%20NBA%20Raw%20Data.csv')

In [3]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg', 'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP',
       'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_s

In [4]:
nba = nba[['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm']]

In [5]:
pd.set_option('display.max_rows',42)
pd.set_option('display.max_columns',42)
nba.head(3).T
#Transpose since it's easier to read

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


### Making a Relational Database

#### First Normal Form

3 Rules (Don't do more than what the rules are saying):

    1. Everytable must have a primary key? Yes, player_id and game_id together.
    
    2. The values inside every cell in every table must be atomic? The Inactives attribute 

In [6]:
nba = nba.drop(['Inactives'],axis=1)
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


    3. There are no repeating groups? No, we don't have.
    
    If you have to split it up and then you get NA's

#### Second Normal Form

Every non-prime columns must depend on the ENTIRE primary key (gameid + playerid) and not just part of the primary key (just gameid or playerid).

We have this in this entity since OT and H_A are not dependent on the player.
If you primary key is just one column, then you can violate this rule. Then, let's do a second player id column! Replace many columns with just one column each.

Replace two columns with just one column which makes it impossible to depend in only it.

In [7]:
nba['game_player_id'] = nba['game_id']+'_'+nba['player_id']
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


1. The data must meet all the criteria to be 1NF,

2. Wikipedia lists the second crition for 2NF as:

    It does not have any non-prime attribute that is functionally dependent on any proper subset of any candidate key of the relation. A non-prime attribute of a relation is an attribute that is not a part of any candidate key of the relation.

#### Third normal form

1. All of the criteria necessary for the database to be in 2NF.

2. “Every non-prime attribute (everything that is not the primary key) … is non-transitively dependent on every [attribute]”

This just means no transitive dependancy. If there is a functional dependency, you have to get rid of it by creating another table.

Calculated column:

-'fg_pct'

-'fg3_pct'

-'ft_pct'

-'trb'

We will delete these

In [8]:
nba = nba.drop(['fg_pct'],axis=1)
nba = nba.drop(['fg3_pct'],axis=1)
nba = nba.drop(['ft_pct'],axis=1)
nba = nba.drop(['trb'],axis=1)
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


Transitive dependencies:

- Some colunmns depend on player
- Some columns depend on game
- game columns depend on team
    - team abbrev
- Some columns depend on team + game
    - Team score

In [9]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'player',
       'player_id', 'starter', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm',
       'game_player_id'],
      dtype='object')

In [10]:
nba_teamgame = nba[['game_id','H_A','Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']]
nba_teamgame = nba_teamgame.drop_duplicates()
nba_teamgame

,game_id,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202204100BRK,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,BRK
12,202204100BRK,H,BRK,134,103.9,0.691,17.9,29.6,0.272,129.0,IND
25,202204100CHO,A,WAS,108,97.7,0.489,8.7,31.5,0.170,110.5,CHO
37,202204100CHO,H,CHO,124,97.7,0.640,15.2,29.7,0.112,126.9,WAS
52,202204100CLE,A,MIL,115,101.9,0.511,10.5,17.4,0.284,112.9,CLE
...,...,...,...,...,...,...,...,...,...,...,...
21491,202112190MIN,H,MIN,111,91.8,0.565,9.0,15.0,0.312,120.9,DAL
21492,202112210DAL,A,MIN,102,93.9,0.538,16.2,20.9,0.215,108.6,DAL
21629,202112280MIN,H,MIN,88,93.1,0.441,9.6,20.4,0.153,94.5,NYK
21708,202112230UTA,A,MIN,116,102.1,0.530,11.6,30.0,0.089,113.7,UTA


In [11]:
nba_game = nba[['game_id','game_date','OT']].drop_duplicates()
nba_game

,game_id,game_date,OT
0,202204100BRK,2022-04-10,0
25,202204100CHO,2022-04-10,0
52,202204100CLE,2022-04-10,0
77,202204100DAL,2022-04-10,0
103,202204100DEN,2022-04-10,1
...,...,...,...
19708,202110300MIN,2021-10-30,0
19726,202112150DEN,2021-12-15,0
19748,202202010MIN,2022-02-01,0
20615,202203270BOS,2022-03-27,0


In [12]:
nba_players = nba[['player_id','player']]
nba_players

,player_id,player
0,halibty01,Tyrese Haliburton
1,hieldbu01,Buddy Hield
2,brissos01,Oshae Brissett
3,jacksis01,Isaiah Jackson
4,mccontj01,T.J. McConnell
...,...,...
31603,gabriwe01,Wenyen Gabriel
31604,gabriwe01,Wenyen Gabriel
31605,wrighmo01,Moses Wright
31606,wrighmo01,Moses Wright


In [13]:
nba_playergame = nba.drop(['player','game_date','OT','H_A','Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev'],axis=1)
nba_playergame

,game_id,player_id,starter,mp,fg,fga,fg3,fg3a,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,off_rtg,def_rtg,bpm,game_player_id
0,202204100BRK,halibty01,1,39:28,7,14,2,5,1,1,2,2,10,2,0,1,0,17,-9,0,0,137,132,1.7,202204100BRK_halibty01
1,202204100BRK,hieldbu01,1,35:53,8,23,5,14,0,0,0,3,6,3,0,2,3,21,0,0,0,94,128,-2.3,202204100BRK_hieldbu01
2,202204100BRK,brissos01,1,35:47,10,20,5,10,3,4,3,5,3,1,0,0,5,28,-9,0,0,137,133,4.4,202204100BRK_brissos01
3,202204100BRK,jacksis01,1,32:01,3,4,0,0,1,2,0,3,0,2,1,2,5,7,3,0,0,89,128,-9.2,202204100BRK_jacksis01
4,202204100BRK,mccontj01,1,30:52,5,15,3,7,1,2,0,3,5,3,0,0,3,14,7,0,0,104,126,-1.7,202204100BRK_mccontj01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31603,202201130NOP,gabriwe01,0,4:26,1,1,1,1,0,0,1,2,0,0,0,2,2,3,-4,0,0,62,110,-6.4,202201130NOP_gabriwe01
31604,202201150SAS,gabriwe01,0,0:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,202201150SAS_gabriwe01
31605,202112220SAC,wrighmo01,0,1:28,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,217,103,24.4,202112220SAC_wrighmo01
31606,202112260LAC,wrighmo01,0,0:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,202112260LAC_wrighmo01


### Our 3rd NF database has 4 tables(entities):

In [14]:
nba_game.head(3)

,game_id,game_date,OT
0,202204100BRK,2022-04-10,0
25,202204100CHO,2022-04-10,0
52,202204100CLE,2022-04-10,0


In [15]:
nba_players.head(3)

,player_id,player
0,halibty01,Tyrese Haliburton
1,hieldbu01,Buddy Hield
2,brissos01,Oshae Brissett


In [16]:
nba_teamgame.head(3)

,game_id,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202204100BRK,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,BRK
12,202204100BRK,H,BRK,134,103.9,0.691,17.9,29.6,0.272,129.0,IND
25,202204100CHO,A,WAS,108,97.7,0.489,8.7,31.5,0.170,110.5,CHO


In [17]:
nba_playergame.head(3)

,game_id,player_id,starter,mp,fg,fga,fg3,fg3a,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,off_rtg,def_rtg,bpm,game_player_id
0,202204100BRK,halibty01,1,39:28,7,14,2,5,1,1,2,2,10,2,0,1,0,17,-9,0,0,137,132,1.7,202204100BRK_halibty01
1,202204100BRK,hieldbu01,1,35:53,8,23,5,14,0,0,0,3,6,3,0,2,3,21,0,0,0,94,128,-2.3,202204100BRK_hieldbu01
2,202204100BRK,brissos01,1,35:47,10,20,5,10,3,4,3,5,3,1,0,0,5,28,-9,0,0,137,133,4.4,202204100BRK_brissos01


### DBMS - Database Management System

Two groups:
    
    1. Sqlite: 
        
        - Save a database as a file in your hardrive
        
    2. mySQL, Postgras, mainDB, Orable, Microsoft:
    
        - Run on memory on a server either on your computer or someone else.

In [18]:
nba_db = sqlite3.connect("nba.db")

In [19]:
nba_game.to_sql('games',nba_db, index = False, chunksize = 1000, if_exists='replace')
nba_players.to_sql('players',nba_db, index = False, chunksize = 1000, if_exists='replace')
nba_teamgame.to_sql('team_game',nba_db, index = False, chunksize = 1000, if_exists='replace')
nba_playergame.to_sql('player_game',nba_db, index = False, chunksize = 1000, if_exists='replace')

31608

In [20]:
myquery = '''
SELECT *
FROM team_game
WHERE Team_Abbrev = 'CLE'
'''
pd.read_sql(myquery, nba_db)

,game_id,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202204100CLE,H,CLE,133,101.9,0.644,9.0,23.3,0.128,130.5,MIL
1,202204080BRK,A,CLE,107,89.6,0.537,8.0,23.8,0.232,119.4,BRK
2,202204050ORL,A,CLE,115,96.8,0.581,10.3,20.5,0.174,118.8,ORL
3,202204030CLE,H,CLE,108,95.0,0.545,8.9,15.6,0.295,113.7,PHI
4,202204020NYK,A,CLE,119,92.1,0.636,9.1,19.4,0.198,129.2,NYK
...,...,...,...,...,...,...,...,...,...,...,...
77,202112300WAS,A,CLE,93,94.2,0.470,11.4,17.8,0.181,98.7,WAS
78,202112310CLE,H,CLE,118,93.2,0.619,8.7,21.1,0.167,126.7,ATL
79,202201020CLE,H,CLE,108,92.8,0.500,7.4,28.6,0.227,116.4,IND
80,202201310CLE,H,CLE,93,88.5,0.482,9.1,16.7,0.143,105.0,NOP


In [21]:
nba_playergame.columns

Index(['game_id', 'player_id', 'starter', 'mp', 'fg', 'fga', 'fg3', 'fg3a',
       'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'plus_minus', 'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg',
       'bpm', 'game_player_id'],
      dtype='object')

In [22]:
# Why does it look like this?
myquery = '''
SELECT *
FROM players
WHERE player
    LIKE 'Kevin%'
ORDER BY player
'''
pd.read_sql(myquery,nba_db)

,player_id,player
0,duranke01,Kevin Durant
1,duranke01,Kevin Durant
2,duranke01,Kevin Durant
3,duranke01,Kevin Durant
4,duranke01,Kevin Durant
...,...,...
394,porteke02,Kevin Porter Jr.
395,porteke02,Kevin Porter Jr.
396,porteke02,Kevin Porter Jr.
397,porteke02,Kevin Porter Jr.


### Joints (How we merge one table with another)

In [23]:
# How do we join the games and team_game tables?

nfl_dict = {'city':['Buffalo','Miami','Boston','New York','Cleveland','Cincinnati',
                       'Pittsburgh','Baltimore','Kansas City','Las Vegas','Los Angeles','Denver',
                       'Nashville','Jacksonville','Houston','Indianapolis','Philadelphia','Dallas',
                       'Washington','Atlanta','Charlotte','Tampa Bay','New Orleans','San Francisco',
                       'Phoenix', 'Seattle','Chicago','Green Bay','Minneapolis','Detroit'],
           'footballteam':['Buffalo Bills','Miami Dolphins','New England Patriots',
                           ['New York Jets', 'New York Giants'],'Cleveland Browns','Cincinnati Bengals',
                          'Pittsburgh Steelers','Baltimore Ravens','Kansas City Chiefs',
                           'Las Vegas Raiders',['L.A. Chargers','L.A. Rams'],'Denver Broncos',
                          'Tennessee Titans','Jacksonville Jaguars','Houston Texans',
                           'Indianapolis Colts','Philadelphia Eagles','Dallas Cowboys',
                           'Washington Bunnyrabbits','Atlanta Falcons','Carolina Panthers',
                           'Tampa Bay Buccaneers','New Orleans Saints', 'San Francisco 49ers',
                           'Arizona Cardinals','Seattle Seahawks','Chicago Bears',
                           'Green Bay Packers','Minnesota Vikings','Detroit Lions']}
nfl_df = pd.DataFrame(nfl_dict)
nfl_df

,city,footballteam
0,Buffalo,Buffalo Bills
1,Miami,Miami Dolphins
2,Boston,New England Patriots
3,New York,"[New York Jets, New York Giants]"
4,Cleveland,Cleveland Browns
5,Cincinnati,Cincinnati Bengals
6,Pittsburgh,Pittsburgh Steelers
7,Baltimore,Baltimore Ravens
8,Kansas City,Kansas City Chiefs
9,Las Vegas,Las Vegas Raiders


In [24]:
nba_dict = {'city':['Boston','New York','Philadelphia','Brooklyn','Toronto',
                   'Cleveland','Chicago','Detroit','Milwaukee','Indianapolis',
                   'Atlanta', 'Washington','Orlando','Miami','Charlotte',
                   'Los Angeles','San Francisco','Portland','Sacramento',
                   'Phoenix','San Antonio','Dallas','Houston','Oklahoma City',
                   'Minneapolis','Denver','Salt Lake City','Memphis','New Orleans'],
           'basketballteam':['Boston Celtics','New York Knicks','Philadelphia 76ers',
                             'Brooklyn Nets','Toronto Raptors',
                            'Cleveland Cavaliers','Chicago Bulls','Detroit Pistons',
                             'Milwaukee Bucks','Indiana Pacers',
                            'Atlanta Hawks','Washington Wizards','Orlando Magic',
                             'Miami Heat','Charlotte Hornets',
                            ['L.A. Lakers','L.A. Clippers'],'Golden State Warriors',
                             'Portland Trailblazers','Sacramento Kings',
                            'Phoenix Suns','San Antonio Spurs','Dallas Mavericks',
                             'Houston Rockets','Oklahoma City Thunder',
                            'Minnesota Timberwolves','Denver Nuggets',
                             'Utah Jazz','Memphis Grizzlies','New Orleans Pelicans']}
nba_df = pd.DataFrame(nba_dict)
nba_df

,city,basketballteam
0,Boston,Boston Celtics
1,New York,New York Knicks
2,Philadelphia,Philadelphia 76ers
3,Brooklyn,Brooklyn Nets
4,Toronto,Toronto Raptors
5,Cleveland,Cleveland Cavaliers
6,Chicago,Chicago Bulls
7,Detroit,Detroit Pistons
8,Milwaukee,Milwaukee Bucks
9,Indianapolis,Indiana Pacers


In [25]:
sportsteams = sqlite3.connect('sportsteams.db')

In [26]:
nba_df.basketballteam = nba_df.basketballteam.astype('string')
nfl_df.footballteam = nfl_df.footballteam.astype('string')

In [27]:
nfl_df.to_sql('nfl',sportsteams,index = False, chunksize = 1000, if_exists='replace')
nba_df.to_sql('nba',sportsteams,index = False, chunksize = 1000, if_exists='replace')

29

In [28]:
myquery = '''
SELECT *
FROM nfl
INNER JOIN nba
    ON nfl.city = nba.city
'''
#Only keeping the ones that have the EXACT same name of cities
#Only in both sets
pd.read_sql(myquery, sportsteams)

,city,footballteam,city,basketballteam
0,Miami,Miami Dolphins,Miami,Miami Heat
1,Boston,New England Patriots,Boston,Boston Celtics
2,New York,"['New York Jets', 'New York Giants']",New York,New York Knicks
3,Cleveland,Cleveland Browns,Cleveland,Cleveland Cavaliers
4,Los Angeles,"['L.A. Chargers', 'L.A. Rams']",Los Angeles,"['L.A. Lakers', 'L.A. Clippers']"
5,Denver,Denver Broncos,Denver,Denver Nuggets
6,Houston,Houston Texans,Houston,Houston Rockets
7,Indianapolis,Indianapolis Colts,Indianapolis,Indiana Pacers
8,Philadelphia,Philadelphia Eagles,Philadelphia,Philadelphia 76ers
9,Dallas,Dallas Cowboys,Dallas,Dallas Mavericks


In [29]:
myquery = '''
SELECT *
FROM nfl f
LEFT JOIN nba b
    ON f.city = b.city
'''
#The LEFT JOIN includes all records from the left side and matched rows from the right table, 
# whereas RIGHT JOIN returns all rows from the right side and unmatched rows from the left table.
# WHERE filters all the rows of dataframe
pd.read_sql(myquery, con = sportsteams)

,city,footballteam,city,basketballteam
0,Buffalo,Buffalo Bills,None,None
1,Miami,Miami Dolphins,Miami,Miami Heat
2,Boston,New England Patriots,Boston,Boston Celtics
3,New York,"['New York Jets', 'New York Giants']",New York,New York Knicks
4,Cleveland,Cleveland Browns,Cleveland,Cleveland Cavaliers
5,Cincinnati,Cincinnati Bengals,None,None
6,Pittsburgh,Pittsburgh Steelers,None,None
7,Baltimore,Baltimore Ravens,None,None
8,Kansas City,Kansas City Chiefs,None,None
9,Las Vegas,Las Vegas Raiders,None,None


### Antijoint 
'''
There are two types of anti joins:
A left anti join : This join returns rows in the left table that have no matching rows in the right table.
A right anti join : This join returns rows in the right table that have no matching rows in the left table.
'''

### Natural Joint
'''
is the same as a joint but with a natural joint you don't have to use ON
'''

### Cross Joint

'''
A cross join, also called a Cartesian product, is a round robin for matching values of the index in one data table to values of the index in the other data table. 
'''

### Who was the high scoring player for each team and each game ?

In [42]:
# For this, we need to use the team_game, player_game, players

myquery = '''
SELECT
    p.player AS player_name,
    t.game_id AS game,
    t.Team_Abbrev AS team,
    pg.pts AS points,
    pg.fg3a
FROM team_game t
INNER JOIN player_game pg
    ON t.game_id = pg.game_id
INNER JOIN players p
    ON pg.player_id = p.player_id
WHERE pg.fg3a > 15 AND pg.fta < 5
ORDER BY points DESC
LIMIT 10 OFFSET 5
'''
pd.read_sql(myquery, nba_db)

# LIMIT 10
# How to sort the rows by points? use ORDER BY (which should come last)


,player_name,game,team,points,fg3a
0,Julius Randle,202203070SAC,NYK,46,16
1,Julius Randle,202203070SAC,NYK,46,16
2,Julius Randle,202203070SAC,NYK,46,16
3,Julius Randle,202203070SAC,NYK,46,16
4,Julius Randle,202203070SAC,NYK,46,16
5,Julius Randle,202203070SAC,NYK,46,16
6,Julius Randle,202203070SAC,NYK,46,16
7,Julius Randle,202203070SAC,NYK,46,16
8,Julius Randle,202203070SAC,NYK,46,16
9,Julius Randle,202203070SAC,NYK,46,16
